# Part 1. Data and Feature Engineering

## Original Dataset

Our primary dataset is sourced from Kaggle:  
[Resale Flat Prices with Distance from Expressway](https://www.kaggle.com/datasets/adisongoh/resale-flat-prices-with-distance-from-expressway).  
It contains information on the resale transactions of Housing & Development Board (HDB) flats in Singapore from 2015 to 2024, comprising over 220,000 observations. The key response variable is `resale_price`, accompanied by 10 predictors:

- `year`: Transaction year (2015–2024)
- `town`: Name of the residential town
- `flat_type`: Flat type (e.g., 1-room, 2-room, 5-room, executive)
- `block`: HDB block number
- `street_name`: Street address
- `storey_range`: Floor range where the flat is located (e.g., 04 TO 06)
- `floor_area_sqm`: Floor area in square meters
- `remaining_lease_years`: Remaining years on the 99-year lease
- `storey_range_category`: Categorized floor range — Low (01–06), Low-Mid (07–12), Mid (13–18), High (19–30), Very High (>30)
- `distance_from_expressway`: Distance to nearest expressway categorized into ranges (≤50m, 51–100m, 101–150m, 151–300m, 301–500m, >500m)

## Integration of MRT/LRT Station Data

We enhanced our original dataset by integrating spatial information about MRT and LRT stations, sourced from the Kaggle dataset:  
[Singapore MRT/LRT Stations with Coordinates](https://www.kaggle.com/datasets/shengjunlim/singapore-mrt-lrt-stations-with-coordinates)

This dataset includes the names and geographic coordinates (latitude and longitude) of all MRT and LRT stations in Singapore, allowing detailed spatial analyses of public transit accessibility.

Using this spatial data, we derived two key accessibility metrics for each HDB flat:

1. **Distance to Nearest Station (within 1 km):** Capturing walkable proximity to public transit.  
2. **Number of Stations within a 1 km radius:** Reflecting the density and accessibility of nearby transit options.

The 1 km threshold was chosen based on urban planning literature, suggesting this distance as a generally acceptable walking range in urban contexts.

In [ ]:
# Check the stations data
# import MRT Stations.csv
import pandas as pd
import numpy as np

# import MRT Stations.csv
df_mrt = pd.read_csv('../data/MRT Stations.csv')
print(df_mrt.head())
print(f"{df_mrt.shape[0]} stations found")

# Check there is LRT data
south_stations = df_mrt[df_mrt['STN_NAME'].str.contains('south', case=False)]
print(south_stations) # Found the LRT stations

   Unnamed: 0  OBJECTID                    STN_NAME STN_NO  \
0           0         1           EUNOS MRT STATION    EW7   
1           1         2  CHINESE GARDEN MRT STATION   EW25   
2           2         3          KHATIB MRT STATION   NS14   
3           3         4          KRANJI MRT STATION    NS7   
4           4         5         REDHILL MRT STATION   EW18   

                                      geometry  Latitude   Longitude  
0  POINT (103.9032524667383 1.319778951553637)  1.319779  103.903252  
1  POINT (103.7325967380734 1.342352820874744)  1.342353  103.732597  
2  POINT (103.8329799077383 1.417383370153547)  1.417383  103.832980  
3  POINT (103.7621654109002 1.425177698770448)  1.425178  103.762165  
4   POINT (103.816816670149 1.289562726402453)  1.289563  103.816817  


In [ ]:
"""
Script to enrich an HDB resale dataset with geographic coordinates using the OneMap Singapore API.

This script reads an HDB resale CSV file and uses the block and street name of each listing to query the OneMap API
for its corresponding latitude and longitude. It merges the coordinates back into the dataset and saves the enriched
data as a new CSV file. The script handles duplicates, API errors, and logs missing coordinates.

Usage:
- Replace `token` with your OneMap API token.
- Ensure the input CSV file contains 'block' and 'street_name' columns.
- Run the script to generate a new file named 'hdb_with_coordinates.csv'.

Dependencies:
- pandas
- requests
- urllib.parse
"""

import pandas as pd
import requests
import urllib.parse

def get_coords_from_block_street(block, street_name, token):
    search_val = f"{block} {street_name}"
    encoded_search_val = urllib.parse.quote(search_val)
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={encoded_search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

    headers = {"Authorization": token}
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code != 200:
            return None
        data = response.json()
        results = data.get("results", [])
        block_str = block.strip().upper()
        filtered = [r for r in results if r.get("BLK_NO", "").upper().startswith(block_str)]
        if not filtered:
            return None
        best_match = filtered[0]
        return {
            "latitude": float(best_match["LATITUDE"]),
            "longitude": float(best_match["LONGITUDE"]),
            "address": best_match.get("ADDRESS", ""),
            "postal": best_match.get("POSTAL", "")
        }
    except Exception as e:
        print(f"Error: {block} {street_name} → {e}")
        return None

def enrich_df_with_coords(df, block_col="block", street_col="street_name", token=None):
    if token is None:
        raise ValueError("You must provide a valid OneMap API token.")

    df = df.copy()

    # 1. Extract unique (block, street_name) pairs.
    unique_locations = df[[block_col, street_col]].drop_duplicates().reset_index(drop=True)
    # Number of unique_locations 
    print(f"Number of unique (block, street_name) pairs: {len(unique_locations)}")

    # 2. Retrieve coordinates for each (block, street_name) pair
    coord_results = []
    for idx, row in unique_locations.iterrows():
        block = row[block_col]
        street = row[street_col]
        result = get_coords_from_block_street(block, street, token)
        if result:
            coord_results.append({
                block_col: block,
                street_col: street,
                **result
            })
        else:
            print(f"Coordinates not found for: {block} {street}")

        if (idx + 1) % 500 == 0 or (idx + 1) == len(unique_locations):
            print(f"Number of processed addresses: {idx + 1} / {len(unique_locations)}")   

    # 3. Create coordinates DataFrame
    df_coords = pd.DataFrame(coord_results)

    # 4. Merge with original DataFrame
    df_merged = pd.merge(df, df_coords, on=[block_col, street_col], how="left")

    return df_merged
	
# Begin execution block
df_hdb = pd.read_csv("../data/hdb_presale_prices_2015-2024_cleaned_regression.csv")

# Replace with your own OneMap token
token = "REPLACE WITH YOUR OWN TOKEN"

# Add coordinates
df_with_coords = enrich_df_with_coords(df_hdb, block_col="block", street_col="street_name", token=token)

# Drop rows with missing coordinates
initial_len = len(df_with_coords)
df_with_coords_cleaned = df_with_coords.dropna(subset=["latitude", "longitude"])
dropped_count = initial_len - len(df_with_coords_cleaned)

print(f"Number of rows dropped due to missing coordinates: {dropped_count}")

# Save CSV file
df_with_coords_cleaned.to_csv("../data/hdb_with_coordinates.csv", index=False)
print("CSV file successfully saved: hdb_with_coordinates.csv")

Result of this code

→ Processed addresses: 9595 / 9595

→ Rows dropped due to missing coordinates: 2505

→ CSV file successfully saved: hdb_with_coordinates.csv

In [ ]:
# Load and inspect the hdb_with_coordinates.csv file.
df_hdb_coords = pd.read_csv("../data/hdb_with_coordinates.csv")
print(df_hdb_coords.shape[0])

df_hdb = pd.read_csv("../data/hdb_presale_prices_2015-2024_cleaned_regression.csv")
print(df_hdb.shape[0])

print(df_hdb_coords.shape[0]/df_hdb.shape[0])

# randomly select 5 rows from df_hdb_coords and shows only block, street_name, latitude, longitude
df_hdb_coords = df_hdb_coords[['block', 'street_name', 'latitude', 'longitude']]
df_hdb_coords.sample(5)

218466
220971
0.9886636707984305


,block,street_name,latitude,longitude
202451,371,CLEMENTI AVENUE 4,1.320575,103.768212
216058,346,CLEMENTI AVENUE 5,1.317017,103.770055
180416,585,PASIR RIS STREET 53,1.375351,103.947377
177894,204,PASIR RIS STREET 21,1.367498,103.962384
106986,107,JALAN BUKIT MERAH,1.279837,103.824758


**Interpretation of results: The number of data points lost during the coordinate matching process is only about 1.1%.**

### Enriches HDB resale data with MRT accessibility information using geographic coordinates.

In [ ]:
"""
Enriches HDB resale data with MRT accessibility information using geographic coordinates.

This script calculates the distance from each unique HDB location to all MRT stations in Singapore
using the haversine formula. It computes:
    - the distance to the nearest MRT station
    - the number of MRT stations within a 1 km walking radius

The results are merged back into the original HDB dataset and saved as a new CSV file.

Inputs:
    - 'hdb_with_coordinates.csv': HDB resale data with latitude and longitude columns
    - 'MRT Stations.csv': MRT station locations with their coordinates

Output:
    - 'hdb_with_mrt_info.csv': the original dataset augmented with MRT proximity features

Dependencies:
    - pandas
    - math
    - tqdm
"""

import pandas as pd
import math
from tqdm import tqdm

# 1. Load data
df_hdb = pd.read_csv("../data/hdb_with_coordinates.csv")
df_mrt = pd.read_csv("../data/MRT Stations.csv")

# Calculate haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of Earth (meter)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

# 2. Extract unique coordinates
unique_coords = df_hdb[['latitude', 'longitude']].drop_duplicates()

# 3. For each unique coordinate, compute MRT information
results = []

for _, coord in tqdm(unique_coords.iterrows(), total=len(unique_coords)):
    lat, lon = coord['latitude'], coord['longitude']
    distances = []

    for _, mrt in df_mrt.iterrows():
        mrt_lat, mrt_lon = mrt['Latitude'], mrt['Longitude']
        dist = haversine_distance(lat, lon, mrt_lat, mrt_lon)
        distances.append((dist, mrt['STN_NAME']))

    # Filter MRT stations within 1km
    close_stations = [d for d in distances if d[0] <= 1000]
    num_nearby = len(close_stations)

    # Find the nearest MRT station
    nearest_station = min(distances, key=lambda x: x[0])
    nearest_name = nearest_station[1]
    nearest_distance = nearest_station[0]

    results.append({
        'latitude': lat,
        'longitude': lon,
        'num_nearby_mrts': num_nearby,
        'nearest_mrt': nearest_name,
        'nearest_mrt_distance': nearest_distance
    })

# 4. Convert results into a DataFrame
df_results = pd.DataFrame(results)

# 5. Merge results back into the original HDB DataFrame
df_hdb = df_hdb.merge(df_results, on=['latitude', 'longitude'], how='left')

print(df_hdb.head(3))

# Save to CSV
df_hdb.to_csv("hdb_with_mrt_info.csv", index=False)

100%|██████████| 9479/9479 [00:20<00:00, 458.44it/s]


   year        town flat_type block           street_name storey_range  \
0  2017  ANG MO KIO    2 ROOM   406  ANG MO KIO AVENUE 10     10 TO 12   
1  2017  ANG MO KIO    3 ROOM   108   ANG MO KIO AVENUE 4     01 TO 03   
2  2017  ANG MO KIO    3 ROOM   602   ANG MO KIO AVENUE 5     01 TO 03   

   floor_area_sqm  remaining_lease_years  resale_price storey_range_category  \
0            44.0                     61      232000.0       Low-Mid (07-12)   
1            67.0                     60      250000.0           Low (01-06)   
2            67.0                     62      262000.0           Low (01-06)   

  distance_from_expressway  latitude   longitude  \
0                    >500m  1.362005  103.853880   
1                    >500m  1.370966  103.838202   
2                    >500m  1.380709  103.835368   

                                             address  postal  num_nearby_mrts  \
0          406 ANG MO KIO AVENUE 10 SINGAPORE 560406  560406                1   
1  108 ANG 